In [28]:
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
import geopandas as gpd
from ipyleaflet import *

# from https://medium.com/analytics-vidhya/interative-map-with-osm-directions-and-networkx-582c4f3435bc

place_name = "Kamppi, Helsinki, Finland"
graph = ox.graph_from_place(place_name)

nodes, edges = ox.graph_to_gdfs(graph)

center = (60.16607, 24.93116)
m = Map(center=center, basemap=basemaps.CartoDB.Positron, zoom=15)

# style for the destination marker
to_marker_style = AwesomeIcon(
    name="circle", icon_color="white", marker_color="red", spin=False
)
from_marker = Marker(location=center)
to_marker = Marker(location=center, icon=to_marker_style)


path_layer_list = []


def handle_change_location(event, marker):
    event_owner = event["owner"]
    event_owner.nearest_node = ox.distance.nearest_nodes(graph, event_owner.location[1], event_owner.location[0])
    marker.nearest_node = ox.distance.nearest_nodes(graph, marker.location[1], marker.location[0])

    try:
        shortest_path = nx.dijkstra_path(
            graph, event_owner.nearest_node, marker.nearest_node, weight="length"
        )
    except:
        return

    if(len(shortest_path) == 1):
        return
    
    if len(path_layer_list) == 1:
        m.remove_layer(path_layer_list[0])
        path_layer_list.pop()

    shortest_path_points = nodes.loc[shortest_path]
    path = gpd.GeoDataFrame(
        [LineString(shortest_path_points.geometry.values)], columns=["geometry"]
    )
    path_layer = GeoData(geo_dataframe=path, style={"color": "black", "weight": 2})
    m.add_layer(path_layer)
    path_layer_list.append(path_layer)

from_marker.observe(lambda event: handle_change_location(event, to_marker), 'location')
to_marker.observe(lambda event: handle_change_location(event, from_marker), 'location')

MultiDiGraph with 3661 nodes and 8763 edges


In [ ]:
def set_nearest_node(marker):
    marker.nearest_node = ox.distance.nearest_nodes(graph, marker.location[0], marker.location[1])

In [ ]:
m.add_layer(from_marker)
m.add_layer(to_marker)
set_nearest_node(from_marker)
set_nearest_node(to_marker)
display(m)